In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

# Get the parent directory of the current file.
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Add the parent directory to the Python path.
sys.path.append(parent_dir)

from ML.preprocessing import create_datasets
from ML.model import make_model

test_batches, train_batches, val_batches = create_datasets("/home/arbersejdiji/inria1358/patches/", 
                                                           batch_size=16, 
                                                           data_size=0.1)

2023-09-25 13:04:32.281571: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 13:04:32.444897: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 13:04:32.461227: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 13:04:36.868216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ Test_batches with 11250 images (704 batches) created
✅ Train_batches with 1518 images (95 batches) created
✅ Val_batches with 3544 images (222 batches) created


In [2]:
#binary cross_entropy
model_ter, history_ter = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='ternausnet',
           epochs =15)

model_seg, history_seg = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='segnet',
           epochs =15)

✅ TernausNet initialized
✅ Model compiled
Epoch 1/2


2023-09-25 13:04:51.840407: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 163840000 exceeds 10% of free system memory.
2023-09-25 13:04:52.033988: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 163840000 exceeds 10% of free system memory.
2023-09-25 13:04:56.556527: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.
2023-09-25 13:04:56.710443: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 245760000 exceeds 10% of free system memory.
2023-09-25 13:04:57.086104: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 245760000 exceeds 10% of free system memory.


50/95 [==============>...............] - ETA: 5:42 - loss: 0.4654 - accuracy: 0.8321 - binary_io_u: 0.0476

KeyboardInterrupt: 

In [9]:
import tensorflow as tf
from tensorflow.keras import backend as K

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())


def jaccard_distance_loss(y_true, y_pred,smooth = 100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [11]:
#Dice Loss
model_dt, history_dt = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='ternausnet',
           loss=jaccard_distance_loss,
           epochs =15)

model_ds, history_ds = make_model(train_batches, 
           val_batches,
           (200,200,3), 
           model_name='segnet',
           loss=jaccard_distance_loss,
           epochs =15)

✅ TernausNet initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 1143s 12s/step - loss: 0.4854 - accuracy: 0.7410 - mean_io_u_4: 0.4208 - val_loss: 0.3729 - val_accuracy: 0.8541 - val_mean_io_u_4: 0.4190
Epoch 2/2
95/95 [==============================] - 1155s 12s/step - loss: 0.3492 - accuracy: 0.8630 - mean_io_u_4: 0.4202 - val_loss: 0.3244 - val_accuracy: 0.8467 - val_mean_io_u_4: 0.4284
✅ Model trained on 95 batches over 2 epochs.
✅ SEGNET Model initialized
✅ Model compiled
Epoch 1/2
95/95 [==============================] - 779s 8s/step - loss: 0.3996 - accuracy: 0.8372 - mean_io_u_5: 0.4770 - val_loss: 0.8035 - val_accuracy: 0.8380 - val_mean_io_u_5: 0.4190
Epoch 2/2
95/95 [==============================] - 778s 8s/step - loss: 0.3431 - accuracy: 0.8716 - mean_io_u_5: 0.4756 - val_loss: 0.7988 - val_accuracy: 0.8380 - val_mean_io_u_5: 0.4190
✅ Model trained on 95 batches over 2 epochs.


In [ ]:
model_ds.predict()